# Utilitários Unificados — Modelos, Datasets e Ruído

Este arquivo reúne três funções principais, com interface unificada e documentação clara:

1. `load_model_unificado(modelo, caminho, ...)`
   - Carrega qualquer modelo: `linear`, `mlp`, `lstm` (Keras) ou `tft` (PyTorch Forecasting).
   - Aceita arquivo/diretório EXATO ou uma pasta raiz para descoberta recursiva.

2. Carregamento de dados (Parquet)
   - `linear/mlp/lstm` → Parquet com `*.meta.json` contendo `x_dim`, `y_dim` (e `seq_len`, `lead` no LSTM).
   - `tft` → Parquet (padrão: retorna DataFrame; opcional: cria `TimeSeriesDataSet`).

3. `add_noise_features(obj, sigma, tipo, ...)`
   - Adiciona ruído GAUSSIANO somente nas FEATURES.
   - `tipo='tfdata'` → aplica em `tf.data.Dataset` (x,y).
   - `tipo='tft'` → aplica em batches de `TimeSeriesDataSet`/`DataLoader` (chaves `encoder_cont`/`decoder_cont`).

> Observação: O notebook foi simplificado para Parquet apenas (sem TFRecords).

## 1) Carregamento Unificado de Modelos

Contrato rápido:
- Entradas:
  - `modelo`: `linear` | `mlp` | `lstm` | `tft`
  - `caminho`: arquivo/diretório exato OU uma pasta para varredura recursiva
  - `prefer_exts` (opcional): lista de extensões a priorizar (ex.: `[".cpfg", ".ckpt"]` para TFT)
  - `allow_unsafe` (bool): permite desserialização insegura apenas para artefatos LOCAIS (Lambda em Keras)
- Saídas: `(obj_modelo, info)`
  - `obj_modelo`: instancia do modelo carregado (Keras ou TemporalFusionTransformer)
  - `info`: dicionário com metadados úteis (`path`, `backend`, `kind`)

In [1]:
# Carregamento dos modelos treinados
import torch
from tensorflow import keras
import json, os

def load_model(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"❌ File not found: {path}")

    ext = os.path.splitext(path)[1].lower()

    # === Load model ===
    if ext == ".keras":
        model = keras.models.load_model(path)
        print("✅ TensorFlow model loaded.")
    elif ext in [".ckpt", ".cptk"]:
        model = torch.load(path, map_location="cpu")
        print("✅ PyTorch Lightning checkpoint loaded.")
    else:
        raise ValueError(f"❌ Unsupported file extension: {ext}")

    # === Load optional JSON config ===
    json_path = f"{os.path.splitext(path)[0]}.model.json"
    config = None
    if os.path.exists(json_path):
        with open(json_path, "r") as f:
            config = json.load(f)
        print(f"🧩 Loaded config: {json_path}")

    return model, config


## Carregando modelo linear
linear, info_linear = load_model('./modelos/treinamento/linear.keras')

## Carregando modelo MLP
mlp, info_mlp = load_model('./modelos/treinamento/mlp.keras')

## Carregando modelo LSTM
lstm, info_lstm = load_model('./modelos/treinamento/lstm.keras')

## Carregando modelo TFT
tft, info_tft = load_model('./modelos/treinamento/TFT/tft/best.ckpt')

model_list = [
    (linear, info_linear),
    (mlp, info_mlp),
    (lstm, info_lstm),
    (tft, info_tft)
]

2025-11-11 11:58:57.934162: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1762873140.061900  859850 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9190 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:06:00.0, compute capability: 8.6
I0000 00:00:1762873140.061900  859850 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9190 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:06:00.0, compute capability: 8.6
/home/victor-bertini/Documentos/tcc_2025/TCC-2025/tfc_venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables wherea

✅ TensorFlow model loaded.
✅ TensorFlow model loaded.
✅ TensorFlow model loaded.
✅ TensorFlow model loaded.


/tmp/ipykernel_859850/1626726152.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(path, map_location="cpu")


✅ PyTorch Lightning checkpoint loaded.


/home/victor-bertini/Documentos/tcc_2025/TCC-2025/tfc_venv/lib/python3.12/site-packages/pytorch_forecasting/models/base/_base_model.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
## 2) Carregando preprocessadores

In [3]:
import glob
import pickle
import tensorflow as tf
from typing import Dict, Any, List, Tuple, Optional

# === LINEAR ===
linear_path = "./data/treinamento/preprocessor/linear_preproc.pkl"
with open(linear_path, "rb") as f:
    linear_preproc = pickle.load(f)


# === LSTM ===
lstm_path = "./data/treinamento/preprocessor/lstm_preproc.pkl"
with open(lstm_path, "rb") as f:
    lstm_preproc = pickle.load(f)

# === TFT ===
tft_path = "./data/treinamento/preprocessor/tft_preproc.pkl"
with open(tft_path, "rb") as f:
    tft_preproc = pickle.load(f)

# === PREPROCESSORS DICT ===
preprocessors = {
    "linear": linear_preproc,
    "mlp": linear_preproc,  # MLP usa o mesmo pré-processador do Linear
    "lstm": lstm_preproc,
    "tft": tft_preproc,
}

print("✅ Todos os preprocessadores carregados com sucesso.")


✅ Todos os preprocessadores carregados com sucesso.


# Funções helper para análise dos modelos

## Funções de coleta de dados

In [4]:

import pandas as pd
from preprocessor import LinearPreprocessor, LSTMPreprocessor, TFTPreprocessor
import numpy as np

def load_dataset_info(model_type: str, dataset_type: str) -> Dict[str, Any]:
    info_path = f'./data/treinamento/{model_type}_dataset_{dataset_type}.meta.json'
    if not os.path.exists(info_path):
        raise FileNotFoundError(f"❌ Dataset info file not found: {info_path}")
    with open(info_path, 'r') as f:
        info = json.load(f)
    return info



def get_problem_df(model_type: str, lag, lead, country_list, problem_name) -> pd.DataFrame:
    # Instanciando preprocessadores
    dataset_info = load_dataset_info(model_type, "test")
    destino_dir = f'./data/{problem_name}'

    if model_type == 'linear':
        df, dataset_info = LinearPreprocessor.load_linear_parquet_dataset(
        data_dir=destino_dir,
        split='test',
        batch_size=256,
        shuffle=True
        )
    elif model_type == 'lstm':
        df,dataset_info = LSTMPreprocessor.load_lstm_parquet_dataset(
        data_dir=destino_dir,
        split='test',
        batch_size=256,
        shuffle=True
        )
    elif model_type == 'tft':
        preproc = TFTPreprocessor(
        model_name="TFT",
        seq_len=lag,
        lead=lead,
        country_list=country_list,
        feature_cols=dataset_info['feature_cols'],
        target_cols=dataset_info['target_cols'],
        data_dir=destino_dir
    )
        df = preproc.load_tft_dataset('test', target_col=dataset_info['target_cols'][0])
    else:
        raise ValueError(f"Modelo desconhecido: {model_type}")
    return df, dataset_info

## Função de Desnormalização e Decoding

In [5]:
# Helper: denormalize + decode using a given Preprocessor instance
from typing import Optional, Dict, Any
import pandas as pd
import numpy as np

def denorm_decode(
    preproc,
    df: Optional[pd.DataFrame] = None,
    normalization_method: Optional[str] = None,
    *,
    decode_time: bool = True,
    decode_label: bool = True,
) -> pd.DataFrame:
    """
    Restaura um DataFrame ao estado original usando os métodos do *preprocessor*:
    - denormalize(): reverte normalização baseada no scaler salvo em preproc.norm_objects
    - decode(): desfaz encodings (label e/ou time_cycle) quando metadados existirem

    Parâmetros
    - preproc: instância de LinearPreprocessor | LSTMPreprocessor | TFTPreprocessor
    - df: DataFrame a restaurar; se None, usa preproc.df_base
    - normalization_method: qual chave usar em preproc.norm_objects; se None, usa a primeira disponível
    - decode_time: se True, tenta decodificar componentes cíclicos de tempo
    - decode_label: se True, tenta decodificar labels categóricos (ex.: country)

    Observações
    - A operação trabalha em preproc.df_base (conforme API da classe). Sempre retorna uma cópia restaurada.
    - Para datasets LSTM salvos como (X,Y) flatten/arrays, a denormalização direta não se aplica aqui; use a denorm nas séries base antes de gerar janelas.
    """
    if df is not None:
        preproc.df_base = df.copy()
    elif getattr(preproc, 'df_base', None) is None or preproc.df_base.empty:
        raise ValueError("df não fornecido e preproc.df_base está vazio.")

    # 1) Denormalize, se houver scaler registrado
    if normalization_method is None:
        # pega a primeira chave disponível (ex.: 'minmax' ou 'standard')
        norm_keys = list((getattr(preproc, 'norm_objects', {}) or {}).keys())
        normalization_method = norm_keys[0] if norm_keys else None

    if normalization_method:
        try:
            preproc.denormalize(normalization_method=normalization_method)
        except Exception as e:
            print(f"[WARN] Falha ao denormalizar com método '{normalization_method}': {e}")
    else:
        print("[INFO] Nenhum método de normalização encontrado no preprocessor; pulando denormalize().")

    # 2) Decodes opcionais
    if decode_label and 'label' in getattr(preproc, 'encod_objects', {}):
        try:
            preproc.decode(encode_method='label')
        except Exception as e:
            print(f"[WARN] Falha ao decodificar labels: {e}")

    if decode_time and 'time_cycle' in getattr(preproc, 'encod_objects', {}):
        try:
            preproc.decode(encode_method='time_cycle', target_col='decoded_datetime')
        except Exception as e:
            print(f"[WARN] Falha ao decodificar time_cycle: {e}")

    return preproc.df_base.copy()

## Funções de avaliação

In [6]:
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import json


def sanity_check_prefetch(ds):
    for x, y in ds.take(1):
        print(np.std(x.numpy(), axis=0))  # per feature std


def avaliar_modelo_keras(
    model,
    dataset,
    titulo="Avaliação do Modelo",
    problem_name="problema",
    max_samples=None,       # None ou <=0 -> usa o máximo possível
    n_leads: int = 1,
    save_dir="./resultados/graficos",
    show_plots: bool = True, # 👈 se False, apenas salva (sem plt.show)
    preproc=None,            # 👈 IGNORADO neste fluxo simplificado
    normalization_method: str | None = None,  # 👈 IGNORADO neste fluxo simplificado
    decode_time: bool = True,                 # 👈 IGNORADO neste fluxo simplificado
    decode_label: bool = True,                # 👈 IGNORADO neste fluxo simplificado
    model_info: dict | None = None,          # 👈 IGNORADO neste fluxo simplificado
    run_denorm_decode: bool = False,         # 👈 desativado e IGNORADO
    salvar_parquet: bool = False,            # 👈 desativado e IGNORADO
):
    """
    Fluxo simplificado (sem denormalizar/decodificar e sem construir DataFrames):
    1) Predict (e evaluate opcional) → obtém Y_pred e Y_real no estado atual (as-is)
    2) Recorta para max_samples
    3) Calcula métricas diretamente em arrays e gera gráficos
    4) NÃO salva datasets (Parquet/CSV) e NÃO realiza denormalização/decodificação
    """
    print(f"🚀 Avaliando modelo '{model.name}' (sem denorm/decode, sem DFs)...")

    # === Avaliação (evaluate) - mantido para registrar métricas de compile ===
    resultados_dict = {}
    try:
        eval_out = model.evaluate(dataset, verbose=0, return_dict=True)
        resultados_dict = {str(k): float(v) for k, v in eval_out.items()}
    except TypeError:
        try:
            resultados = model.evaluate(dataset, verbose=0)
            if not isinstance(resultados, (list, tuple)):
                resultados = [resultados]
            metric_names = getattr(model, "metrics_names", []) or []
            resultados_dict = {str(k): float(v) for k, v in zip(metric_names, resultados)}
        except Exception:
            resultados_dict = {}
    # === Previsões ===
    preds, trues = [], []

    for X_batch, Y_batch in dataset:
        p = model.predict(X_batch, verbose=0)
        preds.append(p)
        trues.append(Y_batch)
    Y_pred = np.concatenate(preds, axis=0)
    Y_real = np.concatenate(trues, axis=0)

    Y_pred = np.squeeze(Y_pred)
    Y_real = np.squeeze(Y_real)

    # Limite de amostras
    if max_samples is None or max_samples <= 0 or max_samples > len(Y_real):
        max_samples = len(Y_real)
    print(f"📏 Utilizando {max_samples} amostras para os gráficos.")
    Y_real = Y_real[:max_samples]
    Y_pred = Y_pred[:max_samples]

    # === Diretório de saída para imagens/JSON de métricas ===
    output_dir = os.path.join(save_dir, problem_name, model.name)
    if os.path.exists(output_dir):
        print(f"🧹 Limpando diretório anterior: {output_dir}")
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    used_scale = 'as-is'

    # === Arrays para métricas/plots (já estão 'as-is') ===
    Y_real_used = Y_real
    Y_pred_used = Y_pred

    # === Métricas gerais (overall) ===
    diff = Y_pred_used - Y_real_used
    abs_diff = np.abs(diff)
    mae_all = float(np.mean(abs_diff))
    mse_all = float(np.mean(diff ** 2))
    rmse_all = float(np.sqrt(mse_all))

    # Correlação de Pearson (overall, vetores achatados)
    y_true_flat = Y_real_used.reshape(-1)
    y_pred_flat = Y_pred_used.reshape(-1)
    if y_true_flat.size > 1 and np.std(y_true_flat) > 0 and np.std(y_pred_flat) > 0:
        corr_all = float(np.corrcoef(y_true_flat, y_pred_flat)[0, 1])
    else:
        corr_all = float('nan')

    ss_res = float(np.sum((y_true_flat - y_pred_flat) ** 2))
    ss_tot = float(np.sum((y_true_flat - np.mean(y_true_flat)) ** 2))
    r2_all = float(1.0 - ss_res / ss_tot) if ss_tot > 0 else float('nan')

    resultados_dict["mae"] = mae_all
    resultados_dict["mse"] = mse_all
    resultados_dict["rmse"] = rmse_all
    resultados_dict["correlacao_pearson"] = corr_all
    resultados_dict["r2"] = r2_all
    resultados_dict["used_scale"] = used_scale

    # Persistir métricas JSON (permanece permitido)
    try:
        with open(os.path.join(output_dir, "metrics.json"), "w", encoding="utf-8") as f:
            json.dump(resultados_dict, f, ensure_ascii=False, indent=2)
        print(f"💾 metrics.json salvo em: {os.path.join(output_dir, 'metrics.json')}")
    except Exception as e:
        print(f"[WARN] Falha ao salvar metrics.json: {e}")

    # === Tabela de métricas gerais (overall_metrics.png) ===
    numeric_items = [(k, v) for k, v in resultados_dict.items() if isinstance(v, (int, float))]
    if numeric_items:
        fig, ax = plt.subplots(figsize=(8, 0.4 * len(numeric_items) + 1))
        ax.axis('off')
        col_labels = ["Métrica", "Valor"]
        table_data = [[k, f"{v:.6f}"] for k, v in numeric_items]
        table = ax.table(cellText=table_data, colLabels=col_labels, loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 1.1)
        ax.set_title(f"{titulo} — Métricas Gerais ({used_scale})", pad=12)
        overall_path = os.path.join(output_dir, "overall_metrics.png")
        plt.tight_layout()
        plt.savefig(overall_path, dpi=150)
        if show_plots:
            plt.show()
        else:
            plt.close()
        print(f"💾 Tabela de métricas gerais salva em: {overall_path}\n")

    # === Métricas por lead + gráficos comparativos ===
    per_lead_metrics = {}
    if isinstance(Y_real_used, np.ndarray) and Y_real_used.ndim > 1:
        effective_leads = min(Y_real_used.shape[1], n_leads)
        for i in range(effective_leads):
            y_t = Y_real_used[:, i]
            y_p = Y_pred_used[:, i]
            d = y_p - y_t
            ad = np.abs(d)
            mae_i = float(np.mean(ad))
            mse_i = float(np.mean(d ** 2))
            rmse_i = float(np.sqrt(mse_i))
            # Correlação de Pearson por lead
            if y_t.size > 1 and np.std(y_t) > 0 and np.std(y_p) > 0:
                corr_i = float(np.corrcoef(y_t, y_p)[0, 1])
            else:
                corr_i = float('nan')
            ss_res_i = float(np.sum((y_t - y_p) ** 2))
            ss_tot_i = float(np.sum((y_t - np.mean(y_t)) ** 2))
            r2_i = float(1.0 - ss_res_i / ss_tot_i) if ss_tot_i > 0 else float('nan')
            per_lead_metrics.setdefault('mae', []).append(mae_i)
            per_lead_metrics.setdefault('mse', []).append(mse_i)
            per_lead_metrics.setdefault('rmse', []).append(rmse_i)
            per_lead_metrics.setdefault('correlacao_pearson', []).append(corr_i)
            per_lead_metrics.setdefault('r2', []).append(r2_i)

        # Gera gráfico comparativo para cada métrica
        def _plot_metric_series(metric_name, values):
            fig, ax = plt.subplots(figsize=(9, 4))
            xs = np.arange(1, len(values) + 1)
            ax.plot(xs, values, marker='o', linewidth=2, color='steelblue')
            ax.set_title(f"{titulo} — {metric_name.upper()} por Lead ({used_scale})")
            ax.set_xlabel("Lead")
            ax.set_ylabel(metric_name.upper())
            ax.grid(True, linestyle='--', alpha=0.6)
            for x, v in zip(xs, values):
                ax.annotate(f"{v:.4f}", (x, v), textcoords="offset points", xytext=(0, -15), ha='center', fontsize=8, color='darkslategray')
            plt.tight_layout()
            fpath = os.path.join(output_dir, f"metric_lead_comparison_{metric_name}.png")
            plt.savefig(fpath, dpi=150)
            if show_plots:
                plt.show()
            else:
                plt.close()
            print(f"💾 Gráfico comparativo por lead salvo: {fpath}")

        for mname, vals in per_lead_metrics.items():
            _plot_metric_series(mname, vals)

    # === Gráficos Predito vs Real por lead (séries) ===
    if isinstance(Y_real_used, np.ndarray) and Y_real_used.ndim > 1:
        effective_leads_series = min(Y_real_used.shape[1], n_leads)
    else:
        effective_leads_series = 1

    for i in range(effective_leads_series):
        y_true_i = Y_real_used[:, i] if (effective_leads_series > 1) else Y_real_used
        y_pred_i = Y_pred_used[:, i] if (effective_leads_series > 1) else Y_pred_used
        plt.figure(figsize=(9, 5))
        plt.plot(y_true_i, label=f"Real lead {i+1}", color="black", linewidth=2, alpha=0.8)
        plt.plot(y_pred_i, label=f"Previsto lead {i+1}", color="tomato", linewidth=2, alpha=0.8)
        plt.title(f"{titulo} - Lead {i+1} ({used_scale})")
        plt.xlabel("Amostra")
        plt.ylabel("Valor")
        plt.legend()
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.tight_layout()
        file_name = f"predito_vs_real_lead{i+1}.png"
        file_path = os.path.join(output_dir, file_name)
        plt.savefig(file_path, dpi=150)
        if show_plots:
            plt.show()
        else:
            plt.close()
        print(f"💾 Gráfico Lead {i+1} salvo em: {file_path}")

    print("✅ Avaliação concluída.")

    result_payload = dict(resultados_dict)
    result_payload['y_true'] = Y_real_used
    result_payload['y_pred'] = Y_pred_used
    if 'per_lead_metrics' in locals() and per_lead_metrics:
        result_payload['per_lead_metrics'] = per_lead_metrics
    return result_payload

# N1A — Série Univariada (seq_len=72, lead=72)


Objetivo
- Prever 24 horas de carga à frente com janelas de 48 horas de histórico para um único país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N1A/linear_dataset_{split}.parquet` + `linear_dataset_{split}.meta.json` → { x_dim, y_dim }
- Parquet (LSTM): `data/N1A/lstm_dataset_{split}.parquet` + `lstm_dataset_{split}.meta.json` → { seq_len=240, lead=72, x_dim, y_dim }


Modelos a comparar
- Linear, MLP, LSTM (Keras) e, opcionalmente, TFT.


Métricas e checks
- MAE, RMSE, MAPE.
- Checar: shapes conforme meta.json; ausência de NaNs; número de amostras > 0.


Visualizações sugeridas
- Boxplot de erro por horizonte; barras de MAE por modelo; curva MAE vs horizonte.


Notas
- As variantes A/B são obtidas reduzindo a janela/horizonte efetivos na avaliação a partir do dataset base (240/72).
- Padding (se houver) deve usar sentinela fixo para permitir mascaramento e ruído seletivo.

In [9]:
# Carregamento dos dataset N1A (Parquet)

# linear/mlp (iguais)
ds_linear, linear_info = get_problem_df(
    model_type='linear',
    lag=72,
    lead=72,
    country_list=['ES'],
    problem_name='N1A'
)

# lstm
ds_lstm, lstm_info = get_problem_df(
    model_type='lstm',
    lag=72,
    lead=72,
    country_list=['ES'],
    problem_name='N1A'
)

tft
ds_tft, tft_info = get_problem_df(
    model_type='tft',
    lag=72,
    lead=72,
    country_list=['ES'],
    problem_name='N1A'
)

## Avaliando dados

### Avaliando modelo Linear
# avaliar_modelo_keras(
#     model=linear,
#     dataset=ds_linear,
#     titulo="Modelo Linear - Problema N1A",
#     problem_name="N1A",
#     n_leads=72,
#     max_samples=1000,
#     show_plots=False,
#     preproc=preprocessors['linear'],
#     model_info=linear_info,           
#     run_denorm_decode=True,           
#     salvar_parquet=True               
# )

# ### Avaliando modelo MLP
# avaliar_modelo_keras(
#     model=mlp,
#     dataset=ds_linear,
#     titulo="Modelo MLP - Problema N1A",
#     problem_name="N1A",
#     n_leads=72,
#     max_samples=1000,
#     show_plots=False,
#     preproc=preprocessors['mlp'],
#     model_info=linear_info,           
#     run_denorm_decode=True,           
#     salvar_parquet=True               
# )

### Avaliando modelo LSTM
avaliar_modelo_keras(
    model=lstm,
    dataset=ds_lstm,
    titulo="Modelo lstm - Problema N1A",
    problem_name="N1A",
    n_leads=72,
    max_samples=1000,
    show_plots=False,
    preproc=preprocessors['lstm'],
    model_info=linear_info,           
    run_denorm_decode=True,           
    salvar_parquet=True               
)

# ### Avaliando modelo TFT
# avaliar_modelo_keras(
#     model=tft,
#     dataset=ds_tft,
#     titulo="Modelo TFT - Problema N1A",
#     problem_name="N1A",
#     n_leads=72,
#     max_samples=1000,
#     show_plots=False,
#     preproc=preprocessors['tft'],
#     model_info=linear_info,           
#     run_denorm_decode=True,           
#     salvar_parquet=True               
# )

📦 TimeSeriesDataSet (test) criado com 3492 amostras.
🚀 Avaliando modelo 'functional' (sem denorm/decode, sem DFs)...


ValueError: Exception encountered when calling Concatenate.call().

[1mDimension 1 in both shapes must be equal, but are 72 and 240. Shapes are [?,72] and [?,240]. for '{{node functional_1/concat_feats_1/concat}} = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32](data, functional_1/emb_repeat_1/Repeat/Reshape_1, functional_1/concat_feats_1/concat/axis)' with input shapes: [?,72,12], [?,240,16], [] and with computed input tensors: input[2] = <-1>.[0m

Arguments received by Concatenate.call():
  • inputs=['tf.Tensor(shape=(None, 72, 12), dtype=float32)', 'tf.Tensor(shape=(None, 240, 16), dtype=float32)']

# N1B — Série Univariada (seq_len=168, lead=48)


Objetivo
- Prever 48 horas de carga à frente com janelas de 168 horas de histórico para um único país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N1B/linear_dataset_{split}.parquet` + meta { x_dim, y_dim }
- Parquet (LSTM): `data/N1B/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (opcional): `data/treinamento/tft_dataset_{split}.parquet`


Modelos a comparar
- Linear, MLP, LSTM, TFT (opcional).


Métricas e checks
- MAE, RMSE, MAPE; validação de shapes e ausência de NaNs.


Visualizações sugeridas
- Barras de MAE médio por modelo; curva de erro por horizonte.


Notas
- As variantes A/B são derivadas do dataset base (240/72) reduzindo janela/horizonte na avaliação, sem retreinar.

# N1C — Série Univariada (seq_len=240, lead=72)


Objetivo
- Pré-treino/treino com a janela de 240 horas e avaliar horizonte de 72 horas.


Artefatos esperados
- Parquet (Linear/MLP): `data/N1C/linear_dataset_{split}.parquet` + meta { x_dim, y_dim }
- Parquet (LSTM): `data/N1C/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (opcional): `data/treinamento/tft_dataset_{split}.parquet`


Modelos a comparar
- Linear, MLP, LSTM, TFT (opcional).


Métricas e checks
- MAE, RMSE, MAPE; número de amostras por split; coerência entre seq_len/lead do meta e shapes efetivos.


Visualizações sugeridas
- Curva comparativa de MAE vs horizonte; top‑k modelos por MAE.


Notas
- Esta variante (C) é a base máxima de lookback e horizonte; A/B são obtidas por redução na avaliação.

# N2A — Múltiplos Países (seq_len=72, lead=24)


Objetivo
- Prever 24 horas com 72 horas de histórico, agrupando por país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N2A/linear_dataset_{split}.parquet` + meta
- Parquet (LSTM): `data/N2A/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (recomendado): `data/treinamento/tft_dataset_{split}.parquet` (colunas: _group_id=country, time_idx crescente por grupo, quantity_MW)


Modelos a comparar
- Linear, MLP, LSTM (podem exigir codificação/flatten por grupo);
- TFT (nativamente multi‑grupo).


Métricas e checks
- MAE/RMSE por país e globais; número de grupos; equilíbrio de amostras por grupo.


Visualizações sugeridas
- Barras de MAE por modelo; facetas por país; curva MAE vs horizonte.


Notas
- As variantes A/B/C partem do dataset base (240/72), aplicando janelas/horizontes reduzidos na avaliação.

# N2B — Múltiplos Países (seq_len=168, lead=48)


Objetivo
- Prever 48 horas com 168 horas de histórico, agrupado por país.


Artefatos esperados
- Parquet (Linear/MLP): `data/N2B/linear_dataset_{split}.parquet` + meta
- Parquet (LSTM): `data/N2B/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (recomendado): `data/treinamento/tft_dataset_{split}.parquet` com `_group_id`, `time_idx`, target.


Modelos a comparar
- Linear, MLP, LSTM; TFT.


Métricas e checks
- MAE/RMSE por país e agregadas; distribuição de amostras por grupo.


Visualizações sugeridas
- Barras de MAE por modelo; linhas por horizonte; painel por país.


Notas
- Variantes A/B/C usam janelas/horizontes efetivos na avaliação; base: 240/72.

# N2C — Múltiplos Países (seq_len=240, lead=72)


Objetivo
- Prever 72 horas com 240 horas de histórico, agrupado por país. Esta variante é a base para reuso em A/B.


Artefatos esperados
- Parquet (Linear/MLP): `data/N2C/linear_dataset_{split}.parquet` + meta
- Parquet (LSTM): `data/N2C/lstm_dataset_{split}.parquet` + meta { seq_len=240, lead=72, x_dim, y_dim }
- TFT (recomendado): `data/treinamento/tft_dataset_{split}.parquet` (agrupado por `_group_id`).


Modelos a comparar
- Linear, MLP, LSTM, TFT.


Métricas e checks
- MAE, RMSE, MAPE; comparação por país; checagem de time_idx e integridade por grupo.


Visualizações sugeridas
- Curva MAE vs horizonte; ranking de modelos por país e global.


Notas
- Base usa seq_len=240 e lead=72; variações A/B podem ser avaliadas reduzindo janela no dataset sem retreino.

# N3 — Robustez a Ruído (sobre N2)

Objetivo
- Medir degradação de desempenho sob ruído gaussiano nas FEATURES (teste), mantendo rótulos intactos.

Configuração
- Conjuntos: use os datasets do N2 (A/B/C).
- Intensidades: σ ∈ {0.00, 0.01, 0.03, 0.05, 0.10}.
- Aplicação:
  - Keras/tf.data: `add_noise_features(ds, sigma, tipo='tfdata', pad_sentinel=-999.0)`.
  - TFT: `add_noise_features(tft_ds ou dataloader, sigma, tipo='tft', batch_size=..., train=False)`.

Métricas e checks
- MAE/RMSE por sigma; checar preservação de sentinela (TF) e invariância de Y.

Visualizações sugeridas
- Curvas MAE vs σ por modelo; heatmap de degradação por horizonte e sigma.

Notas
- Aplique ruído após normalização das features.
- Não altere o treino; apenas avaliação/benchmark.
